# Anotaciones Label Studio

In [1]:
from src.data.coco_class import *

In [2]:
from helper.almaMedical_helper import getDicomPathFiles, getAnnotationDicoms, getAnnotations

In [3]:
path_dataset = './notebooks/dataset/FILESET/'

In [9]:
def create_COCO(dicom_files):
    coco_ds = COCO(set_HIBA=True)
    coco_ds.setInfo(Info(version="1.0.0", description="Spinogram_Dataset"))
    coco_ds.addCategories(Categories(category_id=1, name='spine', supercategory='spine',
                                     keypoints=['C2OT', 'C1AE', 'C1PE', 'C2CE', 'C2AI', 'C2PI', 'C7AS', 'C7PS', 'C7CE',
                                                'C7AI', 'C7PI', 'T1AS', 'T1PS', 'T1CE', 'T1AI', 'T1PI', 'T5AS', 'T5PS',
                                                'T12A', 'T12P', 'L1AS', 'L1PS', 'L4AS', 'L4PS', 'L4AI', 'L4PI', 'S1AS',
                                                'S1MI', 'S1PS', 'F1HC', 'F2HC'],
                                     skeleton=[]))

    annotationNumber = 1

    annotation_dicoms = getAnnotationDicoms(dicom_files)

    for annotation_dicom in annotation_dicoms:

        fileId, fileName, width, height, keypoints = getAnnotations(annotation_dicom, dicom_files)

        cocoImage = ImageCOCO(image_id=fileId, width=width, height=height,
                              file_name=fileName, license_id=coco_ds.getLicense().getId())
        coco_ds.addImage(cocoImage)

        # Create annotation for image
        annotation = Annotation(annotation_id=annotationNumber, image_id=cocoImage.getId(),
                                category_id=1, keypoints=[coordinate for row in keypoints for coordinate in row],
                                num_keypoints=keypoints.shape[0])

        coco_ds.addAnnotation(annotation)

        annotationNumber += 1

    return coco_ds

In [10]:
getDicomPathFiles(path_dataset)

['./notebooks/dataset/FILESET/1036487/0/0/0',
 './notebooks/dataset/FILESET/1036487/0/0/1',
 './notebooks/dataset/FILESET/1036487/0/1/4',
 './notebooks/dataset/FILESET/1036487/0/2/2',
 './notebooks/dataset/FILESET/1036487/0/3/3']

In [11]:
annotation_dicoms = getAnnotationDicoms(getDicomPathFiles(path_dataset))

In [12]:
annotation_dicoms

['./notebooks/dataset/FILESET/1036487/0/1/4']

In [13]:
dataset_test = create_COCO(getDicomPathFiles(path_dataset))

In [14]:
dataset_test.exportJson('./notebooks/dataset/', 'test.json')

# Test de los métodos finalizados

In [6]:
from helper.almaMedical_helper import getDicomPathFiles, create_AlmaMedicalDataset

In [7]:
DICOM_PATH = './data/imagesAlma/'

In [8]:
Dicom_paths = getDicomPathFiles(DICOM_PATH)

In [9]:
AlmaDataset = create_AlmaMedicalDataset(Dicom_paths)

In [10]:
AlmaDataset.exportJson('./data/datasetJson/', 'AlmaDataset.json')

### Prueba con Voxel 51

In [1]:
import fiftyone as fo

In [11]:
# The path to the COCO labels JSON file
labels_path = "./data/datasetJson/AlmaDataset.json"

# Import the dataset
dataset = fo.Dataset.from_dir(
    dataset_type=fo.types.COCODetectionDataset,
    data_path= './data/imagesAlma/JPG/',
    labels_path=labels_path,
    include_id=True,
    label_types=["keypoints"],
    extra_attrs = True,
    classes = ['spine'],
)

 100% |█████████████████████| 2/2 [205.2ms elapsed, 0s remaining, 9.7 samples/s]      


In [12]:
print(dataset)

Name:        2022.09.16.13.42.24
Media type:  image
Num samples: 2
Persistent:  False
Tags:        []
Sample fields:
    id:        fiftyone.core.fields.ObjectIdField
    filepath:  fiftyone.core.fields.StringField
    tags:      fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:  fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    keypoints: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Keypoints)
    coco_id:   fiftyone.core.fields.IntField


In [13]:
dataset

Name:        2022.09.16.13.42.24
Media type:  image
Num samples: 2
Persistent:  False
Tags:        []
Sample fields:
    id:        fiftyone.core.fields.ObjectIdField
    filepath:  fiftyone.core.fields.StringField
    tags:      fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:  fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    keypoints: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Keypoints)
    coco_id:   fiftyone.core.fields.IntField

In [15]:
session = fo.launch_app(dataset)